In [1]:
# size_chains.py
"""Program to down-size option chains
Date: 24-June-2019
Rev: 1.0
"""

from z_helper import *
util.startLoop()

In [2]:
# get the dataframes
df_chains = pd.read_pickle(fspath+'temp_chains.pkl')
df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')

#... Remove chains not meeting put and call std filter

# get dte and remove those greater than maxdte
df_chains = df_chains.assign(dte=df_chains.expiry.apply(get_dte))                    
df_chains = df_chains[df_chains.dte <= maxdte]

# generate std dataframe
df = df_ohlcs[['symbol', 'stDev']]  # lookup dataframe
df = df.assign(dte=df.groupby('symbol').cumcount()) # get the cumulative count for location as dte
df.set_index(['symbol', 'dte'])

df1 = df_chains[['symbol', 'dte']]  # data to be looked at
df2 = df1.drop_duplicates()  # remove duplicates

df_std = df2.set_index(['symbol', 'dte']).join(df.set_index(['symbol', 'dte']))

# join to get std in chains
df_chainstd = df_chains.set_index(['symbol', 'dte']).join(df_std).reset_index()

# make puts and calls dataframe with std filter
df_puts = df_chainstd[df_chainstd.strike < (df_chainstd.undPrice-(df_chainstd.stDev*putstdmult))]
df_puts = df_puts.assign(right = 'P')

df_calls = df_chainstd[df_chainstd.strike > (df_chainstd.undPrice+(df_chainstd.stDev*callstdmult))]
df_calls = df_calls.assign(right = 'C')

df_opt = pd.concat([df_puts, df_calls], sort=False).reset_index(drop=True)

In [36]:
df_puts[(df_puts.symbol=='ACC') & (df_puts.dte==3)]

,symbol,dte,expiry,strike,lot,undPrice,stDev,right
0,ACC,3,20190627,1100.0,400,1533.15,27.808879,P
1,ACC,3,20190627,1120.0,400,1533.15,27.808879,P
2,ACC,3,20190627,1140.0,400,1533.15,27.808879,P
3,ACC,3,20190627,1160.0,400,1533.15,27.808879,P
4,ACC,3,20190627,1180.0,400,1533.15,27.808879,P
5,ACC,3,20190627,1200.0,400,1533.15,27.808879,P
6,ACC,3,20190627,1220.0,400,1533.15,27.808879,P
7,ACC,3,20190627,1240.0,400,1533.15,27.808879,P
8,ACC,3,20190627,1260.0,400,1533.15,27.808879,P
9,ACC,3,20190627,1280.0,400,1533.15,27.808879,P


In [37]:
list(range(-5,0,1))

[-5, -4, -3, -2, -1]

In [38]:
df_puts.groupby(['symbol', 'dte']).nth(list(range(-5,0,1)))

expiry   lot right      stDev  strike  undPrice
symbol    dte                                                   
ACC       3    20190627   400     P  27.808879  1380.0   1533.15
          3    20190627   400     P  27.808879  1400.0   1533.15
          3    20190627   400     P  27.808879  1420.0   1533.15
          3    20190627   400     P  27.808879  1440.0   1533.15
          3    20190627   400     P  27.808879  1460.0   1533.15
          31   20190725   400     P  62.567536  1320.0   1533.15
          31   20190725   400     P  62.567536  1340.0   1533.15
          31   20190725   400     P  62.567536  1360.0   1533.15
          31   20190725   400     P  62.567536  1380.0   1533.15
          31   20190725   400     P  62.567536  1400.0   1533.15
ADANIENT  3    20190627  4000     P   2.435929   120.0    147.00
          3    20190627  4000     P   2.435929   125.0    147.00
          3    20190627  4000     P   2.435929   130.0    147.00
          3    20190627  4000     P   2.435929   135.0    147.00
          3    20190627  4000     P   2.435929   140.0    147.00
          31   20190725  4000     P  16.147407    90.0    147.00
          31   20190725  4000     P  16.147407    95.0    147.00
          31   20190725  4000     P  16.147407   100.0    147.00
          31   20190725  4000     P  16.147407   105.0    147.00
          31   20190725  4000     P  16.147407   110.0    147.00
ADANIPORT 3    20190627  2500     P   1.917355   350.0    403.55
          3    20190627  2500     P   1.917355   360.0    403.55
          3    20190627  2500     P   1.917355   370.0    403.55
          3    20190627  2500     P   1.917355   380.0    403.55
          3    20190627  2500     P   1.917355   390.0    403.55
          31   20190725  2500     P  23.727358   310.0    403.55
          31   20190725  2500     P  23.727358   320.0    403.55
          31   20190725  2500     P  23.727358   330.0    403.55
          31   20190725  2500     P  23.727358   340.0    403.55
          31   20190725  2500     P  23.727358   350.0    403.55
...                 ...   ...   ...        ...     ...       ...
WIPRO     31   20190725  3200     P   5.557174   255.0    286.35
          31   20190725  3200     P   5.557174   260.0    286.35
          31   20190725  3200     P   5.557174   265.0    286.35
          31   20190725  3200     P   5.557174   270.0    286.35
          31   20190725  3200     P   5.557174   275.0    286.35
WOCKPHARM 3    20190627   900     P   5.595813   310.0    366.30
          3    20190627   900     P   5.595813   320.0    366.30
          3    20190627   900     P   5.595813   330.0    366.30
          3    20190627   900     P   5.595813   340.0    366.30
          3    20190627   900     P   5.595813   350.0    366.30
YESBANK   3    20190627  1750     P   3.032532    60.0    109.70
          3    20190627  1750     P   3.032532    70.0    109.70
          3    20190627  1750     P   3.032532    80.0    109.70
          3    20190627  1750     P   3.032532    90.0    109.70
          3    20190627  1750     P   3.032532   100.0    109.70
          31   20190725  1750     P  15.679878    30.0    109.70
          31   20190725  1750     P  15.679878    40.0    109.70
          31   20190725  1750     P  15.679878    50.0    109.70
          31   20190725  1750     P  15.679878    60.0    109.70
          31   20190725  1750     P  15.679878    70.0    109.70
ZEEL      3    20190627  1300     P   6.409307   240.0    346.55
          3    20190627  1300     P   6.409307   260.0    346.55
          3    20190627  1300     P   6.409307   280.0    346.55
          3    20190627  1300     P   6.409307   300.0    346.55
          3    20190627  1300     P   6.409307   320.0    346.55
          31   20190725  1300     P  18.472843   220.0    346.55
          31   20190725  1300     P  18.472843   240.0    346.55
          31   20190725  1300     P  18.472843   260.0    346.55
          31   20190725  1300     P  18.472843   280

In [15]:
df_calls.groupby(['symbol', 'dte']).nth([0,1,2]).head()

expiry  lot right      stDev  strike  undPrice
symbol dte                                                  
ACC    3    20190627  400     C  27.808879  1620.0   1533.15
       3    20190627  400     C  27.808879  1640.0   1533.15
       3    20190627  400     C  27.808879  1660.0   1533.15
       31   20190725  400     C  62.567536  1740.0   1533.15
       31   20190725  400     C  62.567536  1760.0   1533.15